In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.applications.vgg16 import VGG16
from spp.SpatialPyramidPooling import SpatialPyramidPooling
import cv2, numpy as np,pandas as pd
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
import os
import matplotlib.pyplot as plt
from keras import backend as K
import pandas as pd
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection, neighbors)

vgg16_weights_path = '/public/keras_pretrained_model/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
path_lentil = 'Lentil'
path_strawberry = 'Strawberry'
savefile_lentil = 'Lentil_550_data.csv'
savefile_strawberry = 'Strawberry_data.csv'

#function:model construction
#input:weights path
#output:model used to extract features
def new_model(path):
    my_new_model = Sequential()
    my_new_model.add(VGG16(include_top=False, pooling=None, weights=path))
    my_new_model.layers[0].trainable = False
    my_new_model.add(SpatialPyramidPooling([1, 2, 4]))
    my_new_model.add(Dense(4096, activation='relu'))
    return my_new_model

#function:feature extraction
#input:image path
#output:savefile name(path)
def feature_extraction_strawberry(path,savefile):
    vgg_feature_list = []
    name=[]
    cnt=0
    cherry_label=[]
    for im in os.listdir(path):
        img = path + '/' + str(im)
        pos1=im.find('healthy')
        pos2=im.find('leafscorch')
        if pos1 != -1:
            cherry_label.append(0) # 健康
        if pos2 != -1:
            cherry_label.append(1)  #病害
        img = image.load_img(img)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        x = preprocess_input(img)
        vgg_feature = my_new_model.predict(x)
        vgg_feature_np = np.array(vgg_feature)
        vgg_feature_list.append(vgg_feature_np.flatten())
        cnt+=1
        if cnt%100==0:
            print(cnt)
        print('------------ALL DONE!------------')
    vgg_feature = np.array(vgg_feature_list)
    np.savetxt(fname=savefile,X=vgg_feature,delimiter=",")
    
#function:feature extraction
#input:image path
#output:savefile name(path)
def feature_extraction_lentilARR(path,savefile):
    vgg_feature_list = []
    name=[]
    rating=pd.read_csv('Rating.csv')
    cnt=0
    root_rating=[]
    for im in os.listdir(path):
        img = path + '/' + str(im)
        pos2 = im.find('.')
        newname = im[:pos2]
        name.append(newname)
        rate=float(np.extract(rating['Lab_ID']==newname, rating['Root_Rating']))
        root_rating.append(rate)
        img = image.load_img(img)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        x = preprocess_input(img)
        vgg_feature = my_new_model.predict(x)
        vgg_feature_np = np.array(vgg_feature)
        vgg_feature_list.append(vgg_feature_np.flatten())
        cnt+=1
        if cnt%100==0:
            print(cnt)
    print('------------ALL DONE!------------')
    vgg_feature = np.array(vgg_feature_list)
    np.savetxt(fname=savefile,X=vgg_feature,delimiter=",")

if __name__ == "__main__":
    new_model(vgg16_weights_path)
    #feature_extraction_lentilARR(path_lentil,savefile_lentil)
    feature_extraction_strawberry(path_strawberry,savefile_strawberry)